In [ ]:
import pandas as pd
from fastopic import FASTopic
from topmost.preprocess import Preprocess
import pandas as pd
from bertopic import BERTopic
from fastopic import FASTopic
from gensim.models import CoherenceModel
from gensim.corpora.dictionary import Dictionary
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download("punkt")
nltk.download("stopwords")

In [ ]:
df_kotor = pd.read_csv("redacted")
df_kotor.head()

,Unnamed: 0,review_id,review_text,review_rating,review_datetime_utc,review_likes
0,0,ChZDSUhNMG9nS0VJQ0FnSUM4MnFXUlp3EAE,"""must see"" spot, the view is just breathtaking...",5.0,44015.421157,0.0
1,1,ChZDSUhNMG9nS0VJQ0FnSUM2clBQaGJBEAE,-Attention ! Don't make the mistake of giving ...,3.0,44439.762789,1.0
2,2,ChZDSUhNMG9nS0VJQ0FnSURVejhyLVNREAE,..and when you think that it has been hard dug...,5.0,43716.615509,3.0
3,3,ChZDSUhNMG9nS0VJQ0FnSUMwNHM3MUFnEAE,/! \ /! \ More access possible by the outside ...,3.0,43728.446574,1.0
4,4,ChZDSUhNMG9nS0VJQ0FnSURrdllxaUF3EAE,+ 37 ° C in the mountain ........ But the view...,5.0,43651.407384,0.0


In [10]:
preprocess = Preprocess()
model = FASTopic(4, preprocess)

In [ ]:
docs_raw = df_kotor["review_text"].dropna().tolist()

stop_words = set(stopwords.words("english"))
def preprocess(doc):
    tokens = word_tokenize(doc.lower())
    return " ".join([t for t in tokens if t.isalpha() and t not in stop_words])

docs = [preprocess(doc) for doc in docs_raw]
tokenized_docs = [doc.split() for doc in docs]

In [ ]:
fastopic_model = FASTopic(num_topics=8, preprocess=Preprocess())
fastopic_model.fit_transform(df_kotor["review_text"])

fastopic_topic_words = []
for i, x in enumerate(fastopic_model.get_topic_weights()):
    fastopic_topic_words.append([x[0] for x in fastopic_model.get_topic(topic_idx=i)])

parsing texts: 100%|██████████| 1768/1768 [00:00<00:00, 41114.47it/s]
2025-06-14 19:54:56,069 - TopMost - Real vocab size: 3219
2025-06-14 19:54:56,070 - TopMost - Real training size: 1768 	 avg length: 11.697
Training FASTopic: 100%|██████████| 200/200 [00:12<00:00, 15.53it/s]


[['water', 'hike', 'shoes', 'morning', 'sun'],
 ['exercise', 'brilliant', 'fabulous', 'awesome', 'priceless'],
 ['requires', 'physical', 'thousand', 'huge', 'reward'],
 ['charming', 'restaurants', 'lovely', 'romantic', 'stay'],
 ['unforgettable', 'beauty', 'wow', 'super', 'com'],
 ['fortress', 'kotor', 'old', 'town', 'city'],
 ['unforgettable', 'beauty', 'wow', 'super', 'com'],
 ['historical', 'medieval', 'neglected', 'fjord', 'kotorska']]

In [ ]:
fastopic_model = FASTopic(num_topics=10, preprocess=Preprocess(), num_top_words=5)
fastopic_model.fit_transform(df_kotor["review_text"])

# Get topic words
fastopic_topic_words = []
for i, x in enumerate(fastopic_model.get_topic_weights()):
    fastopic_topic_words.append([x[0] for x in fastopic_model.get_topic(topic_idx=i)])
    
dictionary = Dictionary(tokenized_docs)
corpus = [dictionary.doc2bow(text) for text in tokenized_docs]

def evaluate_coherence(topics_words, texts, dictionary, metric='c_v'):
    cm = CoherenceModel(topics=topics_words, texts=texts, dictionary=dictionary, coherence=metric)
    return cm.get_coherence()

def calculate_diversity(topics_words):
    all_words = [word for topic in topics_words for word in topic]
    unique_words = set(all_words)
    return len(unique_words) / len(all_words)

fastopic_coherence = evaluate_coherence(fastopic_topic_words, tokenized_docs, dictionary)
fastopic_diversity = calculate_diversity(fastopic_topic_words)

fastopic_coherence, fastopic_diversity

(0.4573321143375675, 0.82)

In [ ]:
bertopic_model = BERTopic(language="english", calculate_probabilities=False, verbose=False, nr_topics=10, top_n_words=5)
bertopic_topics, _ = bertopic_model.fit_transform(docs)
bertopic_topic_words = [topic for topic in bertopic_model.get_topics().values()]

bert_words = []
for x in bertopic_topic_words:
    new_words = []
    for word, rating in x:
        new_words.append(word)
    bert_words.append(new_words)
    
bertopic_coherence = evaluate_coherence(bert_words, tokenized_docs, dictionary)
bertopic_diversity = calculate_diversity(bert_words)

print(bertopic_coherence, bertopic_diversity)

0.4943371902922772 0.64
